---
<h1 style="text-align: center;">
CSCI 4521: Applied Machine Learning (Fall 2024)
</h1>

<h1 style="text-align: center;">
Homework 3
</h1>

<h3 style="text-align: center;">
(Due Tue, Oct. 29, 11:59 PM CT)
</h3>

---

### The RMS Titanic was a British ocean liner considered by many as "unsinkable." Unfortunately, the Titanic hit an iceberge and sank on April 15, 1912 on her trip from Southampton, England to New York City, USA. There were not enough lifeboards onboard for everyone and, as a result, an estimated 1500 people died out of the 2224 passengers and crew onboard. The Titanic disaster was one of the deadliest ship sinkings. There was a large element of luck involved in surviving the shipwreck but some people were more likely to survive than others.

![rms-titanic-14047.png](attachment:d3b8257e-f179-4545-8953-e44343a5f64d.png)


### In this homework, your task is to predict whether a passenger will survive the shipwreck or not. You need to use machine learning and develop classification models to accomplish this task. The only data you have available is passenger data in the dataset `titanic_dataset_csci4521.csv` which consists of the following features:
- ### Passenger ID,
- ### Ticket class (1 = first class, 2 = second class, 3 = third class),
- ### Passenger name,
- ### Sex,
- ### Age,
- ### Number of siblings or spouses aboard,
- ### Number of parents or children aboard,
- ### Ticket number,
- ### Fare,
- ### Cabin number, and
- ### Port of Embarkation (C = Cherbourg, Q = Queenstown, S = Southampton)
### and label:
- ### Survived ($y_i=1$) or
- ### Not survived ($y_i = 0$).
### You must decide if and how to clean and preprocess the data, which classification algorithms to use, which and how to tune any hyperparameters, how to measure performance, which models to select, and which final model to use.

---

### Write your code here

In [130]:
import numpy as np
import pandas as pd
from scipy.stats import chisquare
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import  chi2,f_classif,SelectKBest

data=pd.read_csv("/content/titanic_dataset_csci4521.csv")


Dropping features that dont have any correlation to data

In [131]:
print("Remove useless features")
data=data.drop(["PassengerId","Name","Ticket"],axis=1)# removed passenger Id  name and Ticket(basically ticket id) as beleive that they have not correlation toward prediciton of surviving
data.head(n=10)

Remove useless features


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3.0,male,22.0,1.0,0,7.2500,NaN,S
1,1,1.0,female,38.0,1.0,0,71.2833,C85,C
2,1,3.0,female,26.0,0.0,0,7.9250,NaN,S
3,1,1.0,female,35.0,1.0,0,53.1000,C123,S
4,0,3.0,male,35.0,0.0,0,8.0500,NaN,S
5,0,3.0,male,NaN,0.0,0,8.4583,NaN,Q
6,0,1.0,male,54.0,0.0,0,51.8625,E46,S
7,0,3.0,male,2.0,3.0,1,21.0750,NaN,S
8,1,3.0,female,27.0,0.0,2,11.1333,NaN,S
9,1,2.0,female,14.0,1.0,0,30.0708,NaN,C


Removing null values

In [132]:
print("null values part")
print("Sum of null values for each columns before removing null columns and rows:\n",data.isnull().sum())
data=data.drop("Cabin",axis=1)# remove cabin as more 75% of the data in cabin is null so cannot actually use column
data["Age"]=data["Age"].replace(np.nan,data["Age"].median())# replace Age null values with median age
data=data.dropna()
print("Sum of null values for each columns after removing all null values:\n",data.isnull().sum())

null values part
Sum of null values for each columns before removing null columns and rows:
 Survived      0
Pclass       11
Sex           5
Age         178
SibSp         1
Parch         0
Fare          0
Cabin       690
Embarked     17
dtype: int64
Sum of null values for each columns after removing all null values:
 Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64


Encoding data sex and embarked

In [133]:
print("before encoding:\n")
print(data.head(n=10),"\n")
label_encoder=LabelEncoder()
data["Sex"]=label_encoder.fit_transform(data["Sex"])
data["Embarked"]=label_encoder.fit_transform(data["Embarked"])
print("data after encoding:\n")
print(data.head(n=10))

before encoding:

   Survived  Pclass     Sex   Age  SibSp  Parch     Fare Embarked
0         0     3.0    male  22.0    1.0      0   7.2500        S
1         1     1.0  female  38.0    1.0      0  71.2833        C
2         1     3.0  female  26.0    0.0      0   7.9250        S
3         1     1.0  female  35.0    1.0      0  53.1000        S
4         0     3.0    male  35.0    0.0      0   8.0500        S
5         0     3.0    male  28.0    0.0      0   8.4583        Q
6         0     1.0    male  54.0    0.0      0  51.8625        S
7         0     3.0    male   2.0    3.0      1  21.0750        S
8         1     3.0  female  27.0    0.0      2  11.1333        S
9         1     2.0  female  14.0    1.0      0  30.0708        C 

data after encoding:

   Survived  Pclass  Sex   Age  SibSp  Parch     Fare  Embarked
0         0     3.0    1  22.0    1.0      0   7.2500         2
1         1     1.0    0  38.0    1.0      0  71.2833         0
2         1     3.0    0  26.0    0.0   

Remove duplicates from data

In [134]:
print("The num data before removing duplicates\n")
print("num samples:",data.shape[0],"\n")
data=data.drop_duplicates()
print("The num data be after removing duplicates\n")
print("num samples:",data.shape[0])

The num data before removing duplicates

num samples: 862 

The num data be after removing duplicates

num samples: 746


Removing negative values from data

In [135]:
print("Data statics before  removing negative valus\n ")
print(data.describe().filter(["min","max"],axis=0),"\n")# Min max for some of the data like parch is quite high so what to normalize data
# some of the data seems to be negative when in fact it cannot be negative such as age and Sibsp so need to filter out  negative values
data=data[(data["Age"]>=0)]
data=data[(data["SibSp"]>=0)]
print("Data statics before  removing negative valus\n")
print(data.describe().filter(["min","max"],axis=0))
scalar=StandardScaler()


Data statics before  removing negative valus
 
     Survived  Pclass  Sex    Age  SibSp   Parch      Fare  Embarked
min       0.0     1.0  0.0  -17.0   -3.0     0.0    0.0000       0.0
max       1.0     3.0  1.0  500.0  300.0  2000.0  512.3292       2.0 

Data statics before  removing negative valus

     Survived  Pclass  Sex     Age  SibSp   Parch      Fare  Embarked
min       0.0     1.0  0.0    0.42    0.0     0.0    0.0000       0.0
max       1.0     3.0  1.0  500.00  300.0  2000.0  512.3292       2.0


Creation new features Family and Rate per person

In [136]:
# Creating new features
print("Creation of new columns for data which include Family and rate per person:\n")
data['Family'] = data.apply(lambda row: row["SibSp"]+row["Parch"]+1,axis=1)
data['Rate_per_person']=data.apply(lambda row: row["Fare"]/row["Family"],axis=1)

#Print the DataFrame after addition of new column
print("New columns of new data family and rate_per_person\n")
print(data[["Family","Rate_per_person"]].head(n=10))

Creation of new columns for data which include Family and rate per person:

New columns of new data family and rate_per_person

   Family  Rate_per_person
0     2.0          3.62500
1     2.0         35.64165
2     1.0          7.92500
3     2.0         26.55000
4     1.0          8.05000
5     1.0          8.45830
6     1.0         51.86250
7     5.0          4.21500
8     3.0          3.71110
9     2.0         15.03540


Removing of outliers from data

In [137]:
df=pd.DataFrame(data, columns=['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch',"Family", 'Fare',"Rate_per_person", 'Embarked'])
df_cols=df.columns[3:9]
df_rows=len(df.index)
print("The number samples before removing the data:\n ")
print("Num samples:",df_rows,"\n")


upperList=[]
lowerList=[]
memory_outlier=[]
outlier_index=[]

for i in df_cols:
  mean=df[i].mean()
  Std=df[i].std()
  upper=mean+3*Std
  lower=mean-3*Std

  outlier_index.append(data[(data[i]>upper)|(data[i]<lower)].index.to_numpy())



outlier_index=np.concatenate(outlier_index)


print("The outliers indexed\n")
outlier_index=np.unique(outlier_index)
print("indexes of outliers", outlier_index,"\n")
print("Total number of outliers removed: ",len(outlier_index),"\n")
for j in outlier_index:
  data=data.drop(index=j,axis=0)


print("Number of samples after removing the outliers\n")
print("Num Samples:",len(data.index),"\n")




The number samples before removing the data:
 
Num samples: 743 

The outliers indexed

indexes of outliers [ 24  27  88 118 131 196 259 270 298 300 312 326 339 343 375 379 382 440
 530 547 560 612 692 703 711 719 733 740 745 782] 

Total number of outliers removed:  30 

Number of samples after removing the outliers

Num Samples: 713 



Standardizing data

In [138]:
print("data before standardizing\n")
print(data.head(n=10),"\n")

scalar=StandardScaler()
data[["Age","SibSp","Parch","Fare","Family","Rate_per_person"]]=scalar.fit_transform(data[["Age","SibSp","Parch","Fare","Family","Rate_per_person"]])# filtered only continuous data to normalize
df=pd.DataFrame(data, columns=['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch',"Family", 'Fare',"Rate_per_person", 'Embarked'])

data=df
print("data after standardizing\n")
print(data.head(n=10))



data before standardizing

   Survived  Pclass  Sex   Age  SibSp  Parch     Fare  Embarked  Family  \
0         0     3.0    1  22.0    1.0      0   7.2500         2     2.0   
1         1     1.0    0  38.0    1.0      0  71.2833         0     2.0   
2         1     3.0    0  26.0    0.0      0   7.9250         2     1.0   
3         1     1.0    0  35.0    1.0      0  53.1000         2     2.0   
4         0     3.0    1  35.0    0.0      0   8.0500         2     1.0   
5         0     3.0    1  28.0    0.0      0   8.4583         1     1.0   
6         0     1.0    1  54.0    0.0      0  51.8625         2     1.0   
7         0     3.0    1   2.0    3.0      1  21.0750         2     5.0   
8         1     3.0    0  27.0    0.0      2  11.1333         2     3.0   
9         1     2.0    0  14.0    1.0      0  30.0708         0     2.0   

   Rate_per_person  
0          3.62500  
1         35.64165  
2          7.92500  
3         26.55000  
4          8.05000  
5          8.45830  


Find importance of features using chi square and anova test

In [139]:

# data has four numerical and three catogorical data will use chi square for catorgorical to catogorical and anova for numerical to catogorical
print("Choosing importance based on any scores which are greater 5 which seem to indicate some level of significance\n")
data_cols_chi=["Pclass","Sex","Embarked"]
data_cols_anova=["Age","SibSp","Parch","Fare","Family","Rate_per_person"]
fs_chi = SelectKBest(score_func = chi2, k = 2 )
fs_anova = SelectKBest(score_func = f_classif, k =4)

target_class=data["Survived"]
main_data_chi=data[data_cols_chi]
main_data_anova=data[data_cols_anova]
fs_chi.fit(main_data_chi,target_class)
main_data_chi=fs_chi.transform(main_data_chi)
fs_anova.fit(main_data_anova,target_class)
for i in range(len(data_cols_chi)):
  print("chi square score for ",data_cols_chi[i],":",fs_chi.scores_[i],"\n")
print("Based on chi square choose Pclass,Sex,Embarked as have score greater than 5", "\n")

for i in range(len(data_cols_anova)):
  print("anova score for ",data_cols_anova[i],":",fs_anova.scores_[i],"\n")
print("Choose anova scores greater than 5 which include Age,Fare,Parch,Rate_per_person\n")
final_data=data[["Pclass","Sex","Fare","Age","Embarked","Parch","Rate_per_person"]]
print("The final data\n")
final_data.head(n=10)








Choosing importance based on any scores which are greater 5 which seem to indicate some level of significance

chi square score for  Pclass : 21.409767378189187 

chi square score for  Sex : 63.654340534721676 

chi square score for  Embarked : 6.766838837905489 

Based on chi square choose Pclass,Sex,Embarked as have score greater than 5 

anova score for  Age : 8.247690321918217 

anova score for  SibSp : 0.6542378540773591 

anova score for  Parch : 6.0226350519515695 

anova score for  Fare : 66.43699016232142 

anova score for  Family : 0.6978116586357108 

anova score for  Rate_per_person : 48.75063629352641 

Choose anova scores greater than 5 which include Age,Fare,Parch,Rate_per_person

The final data



,Pclass,Sex,Fare,Age,Embarked,Parch,Rate_per_person
0,3.0,1,-0.706309,-0.546634,2,-0.487025,-0.770103
1,1.0,0,1.603138,0.610233,0,-0.487025,1.202295
2,3.0,0,-0.681964,-0.257417,2,-0.487025,-0.505200
3,1.0,0,0.947333,0.393321,2,-0.487025,0.642200
4,3.0,1,-0.677456,0.393321,2,-0.487025,-0.497500
5,3.0,1,-0.662730,-0.112809,1,-0.487025,-0.472346
6,1.0,1,0.902701,1.767100,2,-0.487025,2.201586
7,3.0,1,-0.207692,-1.992717,2,0.714528,-0.733756
8,3.0,0,-0.566253,-0.185113,2,1.916081,-0.764799
9,2.0,0,0.116754,-1.125067,0,-0.487025,-0.067161


Training Model

In [140]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn import svm
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,f1_score
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
import seaborn as sns
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier

In [141]:
print(data['Survived'].value_counts())#As the number of data not equally distributed need to use Stratified K fold
print("Values imbalance as number of survived greater that number of not survived")
print("As a result used Stratified k method to ensure equal ")

Survived
0    426
1    287
Name: count, dtype: int64
Values imbalance as number of survived greater that number of not survived
As a result used Stratified k method to ensure equal 


Preparing data for training

In [142]:
X=final_data
y=data['Survived']
X_train, x_test, Y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0,shuffle=True)
X_train = X_train.to_numpy()
y_train = Y_train.to_numpy().ravel() # reshape from 2d to 1d so that sklearn doesn't complain about the shape
X_test = x_test.to_numpy()
y_test = y_test.to_numpy().ravel()

Function used  to Calculate F scores and accuracy

In [143]:
def Get_Accuracy_Scores(model,X_train,X_test,y_train,y_test):
  # Gets f1 and accuracy score
   main_model=model.fit(X_train,y_train)
   y_predict_test=main_model.predict(X_test)
   y_predict_train=main_model.predict(X_train)
   test_accuracy = accuracy_score(y_test, y_predict_test)
   train_accuracy=accuracy_score(y_train,y_predict_train)
   train_f1 = f1_score(y_train, y_predict_train, average="weighted")
   test_f1 = f1_score(y_test, y_predict_test, average="weighted")
   return test_accuracy,train_accuracy,test_f1,train_f1


Random Forrest  model

In [144]:
#Takes 50 second to rund
Stratifiedk_rf=StratifiedKFold(n_splits=10)
rf = RandomForestClassifier(random_state=46,min_samples_leaf=20,max_features=None, n_jobs=-1,min_samples_split=10,oob_score=True)
rf_param_grid={"criterion":["gini", "entropy", "log_loss"],"n_estimators":[20,30,40,50,60,70,80,90,100]}
validation_rf=GridSearchCV(rf,rf_param_grid,cv=Stratifiedk_rf)
fit_rf=validation_rf.fit(X_train,y_train)
rf_model=fit_rf.best_estimator_
test_accuracy,train_accuracy,test_f1,train_f1=Get_Accuracy_Scores(rf_model,X_train,X_test,y_train,y_test)

print("Train Accuracy:", train_accuracy)
print("Test Accuracy:", test_accuracy)
print("Train F1 Score:", train_f1)
print("Test F1 Score:", test_f1)



Train Accuracy: 0.8
Test Accuracy: 0.7902097902097902
Train F1 Score: 0.7913773628557041
Test F1 Score: 0.7788639931497074


Logistic regression model

In [145]:
Stratified_log=StratifiedKFold(n_splits=10)
log = LogisticRegression(random_state=0)
param={"C":[1e-3, 1e-2, 0.1, 1, 10, 1e2, 1e3,10000]}
validation_log = GridSearchCV(log, param,cv=Stratified_log)
fit_log=validation_log.fit(X_train,y_train)
best_model_log=fit_log.best_estimator_
test_accuracy,train_accuracy,test_f1,train_f1=Get_Accuracy_Scores(best_model_log,X_train,X_test,y_train,y_test)

print("Train Accuracy:", train_accuracy)
print("Test Accuracy:", test_accuracy)
print("Train F1 Score:", train_f1)
print("Test F1 Score:", test_f1)



Train Accuracy: 0.7701754385964912
Test Accuracy: 0.7762237762237763
Train F1 Score: 0.7670391679859958
Test F1 Score: 0.7707581172626765


Svm model

In [146]:

svm_model = SVC(random_state=0)
svm_param={"Kernel":["linear", "poly", "rbf", "sigmoid"],"C":[0.001, 0.01, 0.1,0.5,1,10,100,1000,10000],"gamma":["scale","auto"]}
StratifiedSvm=StratifiedKFold(n_splits=10)
validation = GridSearchCV(svm_model, param,cv=StratifiedSvm)

new_score=validation.fit(X_train,y_train)
best_model=new_score.best_estimator_
test_accuracy,train_accuracy,test_f1,train_f1=Get_Accuracy_Scores(best_model,X_train,X_test,y_train,y_test)


print("Train Accuracy:", train_accuracy)
print("Test Accuracy:", test_accuracy)
print("Train F1 Score:", train_f1)
print("Test F1 Score:", test_f1)




Train Accuracy: 0.8228070175438597
Test Accuracy: 0.8041958041958042
Train F1 Score: 0.8184008634978892
Test F1 Score: 0.7967533353171651


K Neibours classiffier model

In [147]:
knn=KNeighborsClassifier(metric="minkowski",metric_params=None, n_jobs=-1)
Stratifiedk_knn=StratifiedKFold(n_splits=10)
param_k={"n_neighbors":[10,20,30,40,50],"weights":["uniform", "distance"],"algorithm":["auto", "ball_tree", "kd_tree", "brute"],"leaf_size":[10,20,30,40,50],"p":[1,2]}
validation = GridSearchCV(knn, param_k,cv=Stratifiedk_knn)
new_score=validation.fit(X_train,y_train)
best_knn=new_score.best_estimator_
test_accuracy,train_accuracy,test_f1,train_f1=Get_Accuracy_Scores(best_knn,X_train,X_test,y_train,y_test)

print("Train Accuracy:", train_accuracy)
print("Test Accuracy:", test_accuracy)
print("Train F1 Score:", train_f1)
print("Test F1 Score:", test_f1)



Train Accuracy: 0.7894736842105263
Test Accuracy: 0.8111888111888111
Train F1 Score: 0.7828503273613155
Test F1 Score: 0.8032993247768444


/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


In [148]:
print("Chose knn as it consitantly has test accuracies in around 0.8 which is quite high with lower train accuracy indicating that it is not over fitting")

Chose knn as it consitantly has test accuracies in around 0.8 which is quite high with lower train accuracy indicating that it is not over fitting


---

### Write your report here

**Data pre processing**

The First thing I did was analyze data for columns which I thought were useless to be dropped which included Name,passenger Id  and ticket which I believed to be more like indexes having no correlation to whether someone survived or not. After dropping uncorrelated columns I checked the data for null values where I realized that most of the cabin contained null values. As a result I decided to drop the cabin. For the case of age I decided to replace it with the median of the data as the null values of age contained around one tenth of the data which is not significant enough to drop the column but is significant enough that dropping the rows may significantly affect the model we want to build. As a result I replaced the null values in age with the median value of the age as the median generally is not affected by the skew of the data. I then removed the rest of the null values using drop Na.I also realized some of the data had some negative values which are wrong values in themselves where the age and sisb columns can only be positive so I removed these primitively as they are not outliers but rather simply incorrect values like null. I then did some feature creation where I added a new column called family which is simply the total siblings columns plus the family plus 1 which includes yourself and a fare per person which is the total family divided by rate. I added this believing that it would help improve the overall accuracy of the model.  After that I removed all outliers which were three standard deviations away from the mean which I got was approximately 30. Then I standardize the data. I initially tried min max scaler but got worse results in my model so I ended up using standard scalar. After standardizing the data I wanted to see the importance of the features however some of the features were categorical and some the features were numerical so instead I used a combination of anova test on  the numerical data and chi square test on categorical data. To choose the feature I simply chose any feature which was able to get higher than 5 which I considered at least marginally significant.




 **Training model**:


To Train the model I decided to use a stratified k fold method to cross validate all my data as I looked at the end data and the ratio of survivors to non survivors was 2:1 which was imbalanced. I chose four models to train the data on which included logistic regression, random forest, support vector machine and kneighborsclassifier. To tune the hyper parameters I initially simply used a for loop to increment through how the model would perform with different hyper parameters however what I realized was if I wanted to tune multiple hyper parameters at the same time I would have to add multiple for loops which would make the overall tuning a hassle so instead a I used gridsearch which take in the model and a grid of hyper parameter with the range of values they could take to tune the model. This however came at the cost of a longer time so as result for random forest  I had to make some compromises on the number of hyper parameters I could choose. To tune hyperparameters I simply went to the documentation of each sklearn function and used them to get a range of hyper parameters. After making the models I wanted to know how well they performed so I used accuracy score and F1 score to figure out on average how well my model was performing.


Result: From the result I believe that Knn is the best model to classify the data. Not only does it have a very high test accuracy and F score accuracy of 0.8 on the test it also has a lower train accuracy indicating that it is not over-fitting to the data.


**Insight gained:**

The main insight I gained was how difficult it is to gain a model of good accuracy as throughout the exercise I had to make many incremental adjustments to improve the quality of the model. For example I had to experiment many times with removing and adding columns and about how significantly it could affect the accuracy. Even when I would make incremental adjustments sometimes the model would become worse and so improving the quality of the model was a game of fine tuning and adjustments.
